In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN

In [ ]:
import numpy as np
import sys, os, string, random

In [ ]:
characters = string.printable
char_indices = dict((c, i) for i, c in enumerate(characters))
indices_char = dict((i, c) for i, c in enumerate(characters))

In [ ]:
INPUT_VOCAB_SIZE = len(characters)
BATCH_SIZE = 200
HIDDEN_SIZE = 100
TIME_STEPS = 3

In [ ]:
def encode_one_hot(line):
  x = np.zeros((len(line), INPUT_VOCAB_SIZE))
  for i, c in enumerate(line):
    index = char_indices[c] if c in characters else char_indices[' ']
    x[i][index] = 1
  return x

In [ ]:
def decode_one_hot(x):
  s = []
  for onehot in x:
    one_index = np.argmax(onehot)
    s.append(indices_char[one_index])
  return ''.join(s)

In [ ]:
def prepare_for_rnn(x):
  ind = [np.array(np.arange(i, i+TIME_STEPS)) for i in range(x.shape[0] - TIME_STEPS + 1)]
  ind = np.array(ind, dtype=np.int32)
  x_rnn = x[ind]
  return x_rnn

In [ ]:
def input_generator(nsamples):
  def generate_line():
    inline = [' ']; outline = []
    for _ in range(nsamples):
      c = random.choice(characters)
      expected = c.lower() if c in string.ascii_letters else ' '
      inline.append(c); outline.append(expected)
    inline.append(' ');
    for i in range(nsamples):
      if outline[i] == ' ': continue
      if i > 0 and i < nsamples-1:
        if outline[i-1] == ' ' and outline[i+1] == ' ':
          outline[i] = ' '
      if (i == 0 and outline[1] == ' ') or (i == nsamples-1 and outline[nsamples-2] == ' '):
        outline[i] = ' '
    return ''.join(inline), ''.join(outline)

  while True:
    input_data, expected = generate_line()
    data_in = encode_one_hot(input_data)
    data_out = encode_one_hot(expected)
    yield prepare_for_rnn(data_in), data_out

In [ ]:
def train(model):
  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  input_gen = input_generator(BATCH_SIZE)
  validation_gen = input_generator(BATCH_SIZE)
  model.fit(input_gen,
            epochs = 50, workers=1,
            steps_per_epoch = 50,
            validation_data = validation_gen,
            validation_steps = 10)

In [ ]:
def build_model():
  model = Sequential()
  model.add(SimpleRNN(HIDDEN_SIZE, input_shape=(None, INPUT_VOCAB_SIZE)))
  model.add(Dense(INPUT_VOCAB_SIZE, activation='softmax'))
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 100)               20100     
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
Total params: 30200 (117.97 KB)
Trainable params: 30200 (117.97 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
train(model)

Epoch 1/50
50/50 [==============================] - 2s 11ms/step - loss: 2.9950 - accuracy: 0.5051 - val_loss: 1.8902 - val_accuracy: 0.5915
Epoch 2/50
50/50 [==============================] - 0s 7ms/step - loss: 1.6624 - accuracy: 0.6213 - val_loss: 1.5822 - val_accuracy: 0.6085
Epoch 3/50
50/50 [==============================] - 0s 7ms/step - loss: 1.4628 - accuracy: 0.5999 - val_loss: 1.2536 - val_accuracy: 0.6215
Epoch 4/50
50/50 [==============================] - 0s 7ms/step - loss: 1.0821 - accuracy: 0.6926 - val_loss: 0.8551 - val_accuracy: 0.7890
Epoch 5/50
50/50 [==============================] - 0s 7ms/step - loss: 0.7228 - accuracy: 0.8497 - val_loss: 0.5566 - val_accuracy: 0.9040
Epoch 6/50
50/50 [==============================] - 0s 8ms/step - loss: 0.4414 - accuracy: 0.9192 - val_loss: 0.3295 - val_accuracy: 0.9585
Epoch 7/50
50/50 [==============================] - 0s 7ms/step - loss: 0.2813 - accuracy: 0.9530 - val_loss: 0.2336 - val_accuracy: 0.9535
Epoch 8/50
50/50 [=

In [ ]:
!wget https://www.gutenberg.org/cache/epub/46144/pg46144.txt

--2024-04-01 19:09:22--  https://www.gutenberg.org/cache/epub/46144/pg46144.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125798 (123K) [text/plain]
Saving to: ‘pg46144.txt.1’

pg46144.txt.1       100%[===================>] 122.85K   463KB/s    in 0.3s    

2024-04-01 19:09:23 (463 KB/s) - ‘pg46144.txt.1’ saved [125798/125798]



In [ ]:
!head pg46144.txt > pg46144_head.txt

In [ ]:
with open('pg46144_head.txt') as f:
  for line in f:
    if line.isspace(): continue
    batch = prepare_for_rnn(encode_one_hot(line))
    preds = model.predict(batch)
    normal = decode_one_hot(preds)
    print(normal, " /// ",line)

2/2 [==============================] - 0s 5ms/step
the project gutenberg ebook of six cups of coffee  ///  ﻿The Project Gutenberg eBook of Six Cups of Coffee

3/3 [==============================] - 0s 3ms/step
his ebook is for the use of anyone anywhere in the united states and  ///  This ebook is for the use of anyone anywhere in the United States and

3/3 [==============================] - 0s 3ms/step
ost other parts of the world at no cost and with almost no restrictions  ///  most other parts of the world at no cost and with almost no restrictions

3/3 [==============================] - 0s 3ms/step
hatsoever  you may copy it  give it away or re use it under the terms  ///  whatsoever. You may copy it, give it away or re-use it under the terms

3/3 [==============================] - 0s 3ms/step
f the project gutenberg license included with this ebook or online  ///  of the Project Gutenberg License included with this ebook or online

3/3 [==============================] - 0s 3ms/ste